# Проект по исследованию базы данных книг

__Описание проекта__

Глобальная пандемия covid-19 внесла изменения во все сферы общественной жизни. С введением локдаунов во многих уголках мира людям пришлось отказаться от повседневных занятий, связанных, как правило, с общественными практиками - посещением публичных пространств и мероприятий. Проводя всё время дома люди, как кажется, стали обращаться к давно забытым практикам. Например, к длинному чтению. Т.е., прочтению не коротких статей в социальных сетях, заметок в Интернете или новостной аналитики на пару-тройку страниц, но книг, требующих вдумчивого и длительного чтения: романов, повестей, литературных циклов. Однако всё это происходит в мире, которыё переживает (или уже пережил), очередной переход к новому технологическому укладу. Теперь читают произведения не на бумажпных носителях, но на электронных - со страниц планшетов, смартфонов, ПК. 

Глобальная пандемия стала благоприятным временем для развития сферы электронных книг. Буйным цветом расцвели стартапы, которые предлагают своим клиентам подобные услуги. Мы не стали исключением и вошли в стартап по предоставлению клиентам электронных книг по подписке. Но что собой представляет этот рынок? Какие кнгиги клиентам нравяться? Какой продукт нам лучше запустить следующим? Чтобы ответить на подобные вопросы и было проведенно данное короткое исследование.

__Цель исследовани__
Сформулировать ценностное предложение для нового продукта

В нашем распоряжении будет несколько датасетов - база данных по книгам, авторам, оценкам, пользователям и издательствам - которые подвергнутся нашей препарации. 

__План работ__

1) Загрузка датасета и его обработка

2) Подсчёт количества книг вышедших после 1 января 2000 года

3) Подсчёт количества обзоров и средней оценки для каждой книги

4) Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

5) Определитние автора с самой высокой средней оценкой книг (с количеством оценок 50 и более)
    
6) Посчёт среднего количества обзоров от пользователей (среди тех, кто поставили больше 50 оценок)

7) Выводы

## 1. Загрузка и обработка датасета

Загрузим необходимые библиотеки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Загрузим датасет

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
query = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


Видим датасет на 1000 строк, без пропусков

In [4]:
query = '''
            SELECT * 
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


Тоже всё нормально

In [5]:
query = '''
            SELECT * 
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


Пропусков нет

In [6]:
query = '''
            SELECT * 
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Всё спокойно

In [7]:
query = '''
            SELECT * 
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


Можем двигаться дальше

## 2. Количество книг, изданных после 1 января 2020 год

In [8]:
query = '''
            SELECT COUNT(book_id) books_count
            FROM books 
            WHERE publication_date >= '01-01-2000'
        '''
pd.io.sql.read_sql(query, con = engine)

,books_count
0,821


С начала 2020 года свет увидела 821 книга

## 3. Количество обзоров и средняя оценка для каждой книги

In [9]:
query = '''
            SELECT b.book_id, b.title, rat.rating_avg, rev.review_count
            FROM books b
            LEFT JOIN (SELECT book_id, AVG(rating) rating_avg FROM ratings GROUP BY book_id) rat ON b.book_id=rat.book_id
            LEFT JOIN (SELECT book_id, COUNT(*) review_count FROM reviews GROUP BY book_id) rev ON b.book_id=rev.book_id
            ORDER BY (rating_avg) DESC
            LIMIT 20
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,rating_avg,review_count
0,913,The Woman in Black,5.0,2
1,459,My Name Is Asher Lev,5.0,2
2,938,Triptych (Will Trent #1),5.0,2
3,518,Pop Goes the Weasel (Alex Cross #5),5.0,2
4,297,Hard Times,5.0,2
5,642,The Big Bad Wolf (Alex Cross #9),5.0,2
6,444,Moneyball: The Art of Winning an Unfair Game,5.0,3
7,469,Neil Gaiman's Neverwhere,5.0,2
8,902,The War of Art: Break Through the Blocks & Win...,5.0,2
9,57,Act of Treason (Mitch Rapp #9),5.0,2


In [10]:
query = '''
            SELECT b.book_id, b.title, rat.rating_avg, rev.review_count
            FROM books b
            LEFT JOIN (SELECT book_id, AVG(rating) rating_avg FROM ratings GROUP BY book_id) rat ON b.book_id=rat.book_id
            LEFT JOIN (SELECT book_id, COUNT(*) review_count FROM reviews GROUP BY book_id) rev ON b.book_id=rev.book_id
            WHERE review_count > 5
            ORDER BY (review_count) DESC
            LIMIT 20
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,rating_avg,review_count
0,948,Twilight (Twilight #1),3.662500,7
1,963,Water for Elephants,3.977273,6
2,750,The Hobbit or There and Back Again,4.125000,6
3,696,The Da Vinci Code (Robert Langdon #2),3.830508,6
4,734,The Glass Castle,4.206897,6
5,497,Outlander (Outlander #1),4.125000,6
6,854,The Road,3.772727,6
7,627,The Alchemist,3.789474,6
8,207,Eat Pray Love,3.395833,6
9,733,The Giver (The Giver #1),3.750000,6


К своему стыду, не узнаю не одной книги из ТОП-20 по рейтингу. 

Из рейтинга книг, у котрых более 5 обзоров, к своему стыду, узнаю "Сумерки", "Кот Да Винчи" (именно так), "Над пропастью во ржи" (её жк запретили в американских школах, разве нет?). Узнаю книги о мальчике-волшебнике со шрамом. И очень рад за Толкиена)))

## 4. Издательство, которое выпустило больше всего книг.

Не могу здесь не повредничать. По условиям задачи, книгой здесь считается нечто от 50 страниц, хотя в Российском ( не знаю как в иностранном) издательском деле принято считать брошюрой печатную продукцию не более 48 страниц...

In [11]:
query = '''
            SELECT p.publisher, COUNT(*) books_count
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY COUNT(*) DESC
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Больше всего книг было выпущено под эгидой издателсьтва Penguin - 42 наименования.

## 5. Автор с самой высокой средней оценкой книг (с количеством оценок 50 и более)

In [12]:
query = '''
            WITH
            rating AS (SELECT book_id, AVG(rating) ratings
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(*) > 50
            ) 
            SELECT a.author, AVG(ratings) rating_avg
            FROM rating JOIN (SELECT book_id, author_id FROM books) b ON rating.book_id=b.book_id
            JOIN (SELECT author_id, author FROM authors) a ON b.author_id=a.author_id
            GROUP BY a.author
            ORDER BY avg(ratings) DESC
        '''
pd.io.sql.read_sql(query, con = engine)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самый популярный автор - Джоан Роулинг - что, в принципе ожидаемо. Отметим, что достаточно много лёгкой, фэтезийной литературы, с помощью которой люди пытаются сбежать от тягот и невзгод реального мира...

## 6. Cреднее количество обзоров от пользователей (среди тех, кто поставили больше 50 оценок)

In [16]:
query = ''' 
            WITH 
            usr AS (SELECT username, COUNT(rating_id)
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50)
            
            SELECT AVG(count_reviews) avg_n_review
            FROM (
            SELECT username, COUNT(review_id) count_reviews
            FROM reviews
            WHERE username IN (SELECT username from usr )
            GROUP BY username
            ) review

        '''
pd.io.sql.read_sql(query, con = engine)

,avg_n_review
0,24.333333


Те, кто много читают, много и пишут. Графомания заразна.

## Выводы

1) С 1-го января 2020 года была издана 821 рукопись.

2) Самое "массовое" издание Penguin, под лейблом которого (Penguin Books и Penguin Classic) вышло 6,5% от всех книг в датасете. 

3) Потребители предпочитают литературу о вымышленном мире. Зачастую ориентированную изначально на детей/подростков.

4) Активные в чтении пользователи, также активны и оценки прочитанного. 